[ References ]
* https://www.deeplearning.ai/short-courses/chatgpt-prompt-engineering-for-developers/

# Prompt Engineering for Developers

**프롬프트(Prompts)** 에는 원하는 작업을 수행하기 위해 언어 모델에 전달되는 지시(instruction) 및 문맥/컨텍스트(context)가 포함됩니다.
<br>

**프롬프트 엔지니어링(Prompt engineering)** 은 다양한 애플리케이션에서 언어 모델(LM)을 효율적으로 사용하기 위해 프롬프트를 개발하고 최적화하는 작업입니다.

```
(ChatGPT)  LLM에서의 프롬프트 엔지니어링이란? 한 문장으로

LLM에서의 프롬프트 엔지니어링은 사용자가 제공한 텍스트 입력에 기반하여 모델의 출력을 조절하고 개선하는 기술입니다.
```

#### Contents

* Guidelines for Prompting
  - Write clear and specific instructions 
  - Give the model time to "think"
  
* Iterative prompt development
* Capabilities 
  - Summarizing
  - Infering
  - Transforming
  - Expanding
* Building a Chatbot

# Guidelines for Prompting
이 단원에서는 대규모 언어 모델에 대한 효과적인 프롬프트를 작성하기 위해 두 가지 프롬프트 원칙과 관련 전술을 연습합니다.

## Setup
#### API 키와 관련 Python 라이브러리를 로드합니다.

In [1]:
#!pip install openai
#!pip install -U python-dotenv
import openai
import os

from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())  # read local .env file of OPENAI_API_KEY='sk-***'
openai.api_key  = os.getenv('OPENAI_API_KEY')

#### helper function
이 과정에서는 OpenAI의 `gpt-3.5-turbo` 모델과 [chat completions endpoint](https://platform.openai.com/docs/guides/chat)를 사용합니다.

이 도우미 함수를 사용하면 프롬프트를 더 쉽게 사용하고 생성된 출력을 볼 수 있습니다.

In [5]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.chat.completions.create(
        model=model,
        messages=messages,
        temperature=0, # 이것는 모델 출력의 무작위성 정도를 말합니다.[0~1]
    )
    return response.choices[0].message.content

## Prompting Principles
- **Principle 1: 명확하고 구체적인 지침 작성**
- **Principle 2: 모델에게 "생각"할 시간을 주십시오.**

### **Principle 1: 명확하고 구체적인 지침 작성**
* Tactic 1: 구분 기호(delimiters)를 사용하여 입력의 고유한 부분을 명확하게 표시
* Tactic 2: 구조화된 출력 요청
* Tactic 3: 모델에게 조건 충족 여부를 확인하도록 요청
* Tactic 4: "Few-shot" prompting

#### Tactic 1: 구분 기호(delimiters)를 사용하여 입력의 고유한 부분을 명확하게 표시
- Delimiters can be anything like: 
  - triple backticks: ```, 
  - triple quotes: """, 
  - angle brackets: < >, 
  - XMP tags: `<tag> </tag>`,
  - colon: `:`

In [6]:
text = f"""
가능한 한 명확하고 구체적인 지침을 제공하여 \
모델이 수행할 작업을 표현해야 합니다.\
이렇게 하면 모델이 원하는 출력으로 안내되고 \
관련이 없거나 잘못된 응답을 받을 가능성이 줄어듭니다. \
명확한 프롬프트를 작성하는 것과 짧은 프롬프트를 작성하는 것을\
혼동하지 마십시오.
대부분의 경우 프롬프트가 길면 길수록 모델에 대한 
명확성과 컨텍스트를 제공하여 보다 상세하고 관련성 높은 출력으로\
이어질 수 있습니다.
"""
prompt = f"""
세 개의 백틱으로 구분된 텍스트를 짧은 한 문장으로 요약합니다.
```{text}```
"""
response = get_completion(prompt)
print(response)

모델이 수행할 작업을 명확하고 구체적으로 지시해야 하며, 프롬프트가 길면 더 많은 컨텍스트를 제공하여 더 관련성 높은 출력을 얻을 수 있습니다.


OpenAI 의 API 사용이 불가한 경우에는 다음의 프롬프트를 복사해서 ChatGPT-3.5 웹인터페이스의 프롬프트 입력창에 입력해도 됩니다. 

In [7]:
prompt

'\n세 개의 백틱으로 구분된 텍스트를 짧은 한 문장으로 요약합니다.\n```\n가능한 한 명확하고 구체적인 지침을 제공하여 모델이 수행할 작업을 표현해야 합니다.이렇게 하면 모델이 원하는 출력으로 안내되고 관련이 없거나 잘못된 응답을 받을 가능성이 줄어듭니다. 명확한 프롬프트를 작성하는 것과 짧은 프롬프트를 작성하는 것을혼동하지 마십시오.\n대부분의 경우 프롬프트가 길면 길수록 모델에 대한 \n명확성과 컨텍스트를 제공하여 보다 상세하고 관련성 높은 출력으로이어질 수 있습니다.\n```\n'

구분 기호를 사용하는 것은 또한 **프롬프트 주입(prompt injection)** 을 피하는 데 유용한 기술입니다. 프롬프트 주입이 사용자가 프롬프트에 일부 입력을 추가할 수 있는 경우 모델에 일종의 충돌 지침을 제공하여 당신이 원하는 대로 하기보다는 사용자의 지시를 따르도록 만드는 것을 말합니다.

<img src="https://static.simonwillison.net/static/2023/prompt-injection-delimiters-bad-fix.jpg" width=450>

#### Tactic 2: 구조화된 출력 요청
- JSON, HTML

In [8]:
prompt = f"""
저자 및 장르와 함께 3개의 가상의 책 제목 목록을 생성합니다. \
다음 키와 함께 JSON 형식으로 제공하십시오: \
책_아이디, 제목, 저자, 장르.
"""
response = get_completion(prompt)
print(response)

{
    "도서목록": [
        {
            "책_아이디": 1,
            "제목": "시간을 달리하는 소녀",
            "저자": "김영하",
            "장르": "소설"
        },
        {
            "책_아이디": 2,
            "제목": "별을 쫓는 소년",
            "저자": "이도우",
            "장르": "판타지"
        },
        {
            "책_아이디": 3,
            "제목": "빛의 여왕",
            "저자": "박민경",
            "장르": "로맨스"
        }
    ]
}


#### Tactic 3: 모델에게 조건 충족 여부를 확인하도록 요청

In [9]:
text_1 = f"""
차 한 잔을 만드는 것은 쉽습니다! 먼저, 물을 끓일 필요가 있습니다.\
그런 일이 일어나는 동안 컵을 들고 그 안에 티백을 넣으십시오.\
물이 충분히 뜨거우면 티백 위에 붓기만 하면 됩니다.\
차를 우려낼 수 있도록 잠시 그대로 두십시오. \
몇 분 후 티백을 꺼냅니다. 기호에 따라 설탕이나 우유를\
약간 첨가하여 맛을 볼 수 있습니다. 그리고 그게 다입니다!\
맛있는 차 한 잔을 즐길 수 있습니다.
"""
prompt = f"""
삼중 따옴표로 구분된 텍스트가 제공됩니다.
일련의 지침이 포함된 경우 해당 지침을 다음 형식으로 다시 작성하십시오.

1 단계 - ...
2 단계 - …
…
N 단계 - …

텍스트에 일련의 지침이 포함되어 있지 않으면\
\"제공된 단계 없음\"이라고 쓰면 됩니다.
\"\"\"{text_1}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 1:")
print(response)

Completion for Text 1:
1 단계 - 물을 끓입니다.
2 단계 - 컵에 티백을 넣습니다.
3 단계 - 뜨거운 물을 티백 위에 붓습니다.
4 단계 - 차를 우려냅니다.
5 단계 - 티백을 꺼냅니다.
6 단계 - 설탕이나 우유를 첨가하여 맛을 본 후 즐깁니다.


In [10]:
text_2 = f"""
오늘은 태양이 밝게 빛나고 새들이 지저귀고 있습니다.\
공원에서 산책하기 좋은 날입니다. 꽃은 피고 나무는\
바람에 부드럽게 흔들립니다. 사람들은 아름다운 날씨를\
즐기며 여기저기 돌아다니고 있습니다. 누군가는 소풍을 가고,\
다른 누군가는 게임을 하거나 잔디밭에서 휴식을 취하고 있습니다.\
야외에서 시간을 보내고 자연의 아름다움을 감상하기에 완벽한 날입니다.
"""
prompt = f"""
삼중 따옴표로 구분된 텍스트가 제공됩니다.
일련의 지침이 포함된 경우 해당 지침을 다음 형식으로 다시 작성하십시오.

1 단계 - ...
2 단계 - …
…
N 단계 - …

텍스트에 일련의 지침이 포함되어 있지 않으면\
\"제공된 단계 없음\"이라고 쓰면 됩니다.
\"\"\"{text_2}\"\"\"
"""
response = get_completion(prompt)
print("Completion for Text 2:")
print(response)

Completion for Text 2:
제공된 단계 없음


#### Tactic 4: "Few-shot" prompting
작업을 성공적으로 완료한 사례를 제시하십시오. <br>
그런 다음 모델에게 작업을 수행하도록 요청합니다.

In [11]:
prompt = f"""
당신의 임무는 일관된 스타일로 대답하는 것입니다.

<아이>: 인내심에 대해 가르쳐 주세요.

<할아버지>: 가장 깊은 계곡을 파고드는 강은 겸손한 샘에서 흘러나온단다.\
가장 웅장한 교향곡은 하나의 음표에서 비롯된단다. \
가장 복잡한 태피스트리는 하나의 실에서 시작된단다.

<아이>: 탄력성에 대해 가르쳐 주세요.
"""
response = get_completion(prompt)
print(response)

<할아버지>: 탄력성은 어려운 시련을 겪을 때 우리가 얼마나 빨리 회복하는지를 말합니다. 마치 유연한 나무가 강한 바람에 휘어지지만 다시 일어설 수 있는 것처럼, 우리도 어려움을 극복하고 다시 일어날 수 있어야 합니다. 이는 우리의 내면 강함과 인내심을 키우는 것에 도움이 됩니다.


### Principle 2: Give the model time to “think” 
* Tactic 1: 작업을 완료하는 데 필요한 단계를 지정
* Tactic 2: 성급하게 결론을 내리기 전에 모델이 자체 솔루션을 내도록 지시합니다.

#### Tactic 1: 작업을 완료하는 데 필요한 단계를 지정

* Step 1: ...
* Step 2: ... <br>
...
* Step N: ...

In [12]:
text = f"""
매력적인 마을에서 Jack과 Jill 남매는 언덕 위 우물에서 물을 길러 나갔습니다.\
그들이 즐겁게 노래하며 올라가는 동안 불행이 닥쳤습니다.\
잭이 돌에 걸려 넘어져 언덕 아래로 굴러 떨어졌고 질도 그 뒤를 따랐습니다.\
약간 타박상을 당했지만 두 사람은 위로의 포옹과 함께 집으로 돌아 왔습니다.\
사고에도 불구하고 그들의 모험심은 흐트러지지 않았으며\
그들은 기쁨으로 탐험을 계속했습니다.
"""
# example 1
prompt_1 = f"""
다음 작업을 수행합니다.
1 - 세 개의 백틱으로 구분된 다음 텍스트를 한 문장으로 요약합니다.
2 - 요약된 한 문장을 프랑스어로 번역합니다.
3 - 프랑스어 요약에 각 이름을 나열합니다.
4 - french_summary, num_names 키가 포함된 json 개체를 출력합니다.

답을 줄 바꿈으로 구분하십시오.

텍스트:
```{text}```
"""
response = get_completion(prompt_1)
print("Completion for prompt 1:")
print(response)

Completion for prompt 1:
Jack과 Jill은 우물에서 물을 길러 나갔지만 불행이 닥쳤습니다. 두 사람은 사고 후에도 모험심이 흐트러지지 않았고 기쁨으로 탐험을 계속했습니다.

Jack et Jill, 우물에서 물을 길러 나갔지만 불행이 닥쳤습니다. 두 사람은 사고 후에도 모험심이 흐트러지지 않았고 기쁨으로 탐험을 계속했습니다.

{
  "french_summary": "Jack et Jill, 우물에서 물을 길러 나갔지만 불행이 닥쳤습니다. 두 사람은 사고 후에도 모험심이 흐트러지지 않았고 기쁨으로 탐험을 계속했습니다.",
  "num_names": ["Jack", "Jill"]
}


#### Ask for output in a specified format

In [13]:
prompt_2 = f"""
귀하의 임무는 다음 작업을 수행하는 것입니다.
1 - <>로 구분된 다음 텍스트를 한 문장으로 요약하십시오.
2 - 요약된 한 문장을 프랑스어로 번역합니다.
3 - 프랑스어 요약에 각 이름을 나열합니다.
4 - 다음을 포함하는 json 개체를 출력합니다.
   다음 키: french_summary, num_names.
   
다음 형식을 사용하십시오.:
Text: <text to summarize>
Summary: <summary>
Translation: <summary translation>
Names: <list of names in Italian summary>
Output JSON: <json with summary and num_names>

Text: <{text}>
"""
response = get_completion(prompt_2)
print("\nCompletion for prompt 2:")
print(response)


Completion for prompt 2:
Summary: Jack과 Jill은 우물에서 물을 길러 나갔지만 불행히도 넘어져서 다쳤지만, 포옹하며 집으로 돌아왔습니다.

Translation: Jack et Jill sont allés chercher de l'eau au puits dans un charmant village, mais ont malheureusement trébuché et se sont blessés, mais sont rentrés chez eux en se serrant dans les bras.

Names: Jack, Jill

Output JSON: 
{
  "french_summary": "Jack et Jill sont allés chercher de l'eau au puits dans un charmant village, mais ont malheureusement trébuché et se sont blessés, mais sont rentrés chez eux en se serrant dans les bras.",
  "num_names": 2
}


#### Tactic 2: 성급하게 결론을 내리기 전에 모델이 자체 솔루션을 내도록 지시합니다.

In [14]:
prompt = f"""
학생의 솔루션이 올바른지 확인하십시오.

질문:
저는 태양광 발전 시설을 짓고 있는데 재정 문제를 해결하는 데 \
도움이 필요합니다.
- 토지 비용 $100/평방피트
- 평방피트당 $250에 태양광 패널을 살 수 있습니다.
- 연간 $100,000의 고정 비용과 평방 피트당 $10의 추가 비용이 드는\
유지 보수 계약을 협상했습니다.
평방 피트 수의 함수로 운영 첫 해의 총 비용은 얼마입니까?

학생의 솔루션:
x를 평방 피트 단위의 설치 크기라고 합니다.
비용:
1. 토지 비용: 100x
2. 태양 전지 패널 비용: 250x
3. 유지비 : 100,000 + 100x
총 비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
"""
response = get_completion(prompt)
print(response)

학생의 솔루션은 올바릅니다. 총 비용은 450x + 100,000이 맞습니다.


학생의 솔루션은 실제로 정확하지 않습니다.
먼저 모델이 자체 솔루션을 내도록 지시하여 이 문제를 해결할 수 있습니다.

In [15]:
prompt = f"""
귀하의 임무는 학생의 솔루션이 올바른지 여부를 결정하는 것입니다.
문제를 해결하려면 다음을 수행하십시오.
- 먼저 문제에 대한 자신의 솔루션을 찾으십시오.
- 그런 다음 자신의 솔루션을 학생의 솔루션과 비교하고 \
학생의 솔루션이 올바른지 평가하십시오.
문제를 직접 해결 해보기 전에는 학생의 솔루션이 맞는지 판단하지 마십시오.

다음 형식을 사용하시오:
질문:
```
질문은 여기에.
```
학생의 솔루션:
```
학생의 솔루션을 여기에 둡니다.
```
실제 솔루션:
```
솔루션을 해결하는 단계이며, 여기에 솔루션을 둡니다.
```
학생의 솔루션이 방금 계산한 자신의 솔루션과 동일합니까?
```
예 또는 아니오
```
Student grade:
```
correct or incorrect
```

질문:
```
저는 태양광 발전 시설을 짓고 있는데 재정 문제를 해결하는 데 도움이 필요합니다.
- 토지 비용 $100/평방피트
- 평방피트당 $250에 태양광 패널을 살 수 있습니다.
- 연간 $100,000의 고정 비용과 평방 피트당 $10의 추가 비용이 드는 유지 보수 계약을 협상했습니다.
평방 피트 수의 함수로 운영 첫 해의 총 비용은 얼마입니까?
``` 
학생의 솔루션:
```
x를 평방 피트 단위의 설치 크기라고 합니다.
비용:
1. 토지 비용: 100x
2. 태양 전지 패널 비용: 250x
3. 유지비 : 100,000 + 100x
총 비용: 100x + 250x + 100,000 + 100x = 450x + 100,000
```
실제 솔루션:
"""
response = get_completion(prompt)
print(response)

```
총 비용 = (100 * x) + (250 * x) + 100,000 + (10 * x)
총 비용 = 360x + 100,000
```
학생의 솔루션이 방금 계산한 자신의 솔루션과 동일합니까?
```
아니오
```
Student grade:
```
incorrect
```
